# Projet Data Visualisation
## Transformation des données pour leur utilisation

In [49]:
import pandas as pd
import unidecode as ud # pour enlever les accents aux libellés des colonnes
import re

### Lecture des données

In [50]:
file = "releves_bancaires_2021.csv"
df_operations = pd.read_csv(f"../data/{file}", sep = ";", decimal = ",")[["Libellé", "Date", "Débit euros", "Catégorie"]]
df_operations = df_operations.loc[df_operations["Débit euros"].notnull ()] # on ne garde que les débits

In [51]:
df_operations.head()

,Libellé,Date,Débit euros,Catégorie
1,PRELEVEMENT \r\nBouygues Telecom,27/12/2021,11.99,Téléphone
2,PAIEMENT PAR CARTE \r\nLA FRANCAISE DES...,24/12/2021,8.50,Jeux
4,PAIEMENT PAR CARTE \r\nAMAZON EU SARL P...,23/12/2021,79.72,Cadeaux
5,PAIEMENT PAR CARTE \nFNAC LYON CEDEX 0 ...,20/12/2021,68.98,Cadeaux
6,PAIEMENT PAR CARTE \nGLOCAL DIFFUSION LUN...,20/12/2021,25.00,Cadeaux


In [52]:
df_expenses_cat = pd.read_excel("../data/expenses_by_category.xlsx").rename(columns={"Unnamed: 0": "Catégorie", "Autres inactifs":"Etudiants", "Ensemble":"Français"})[["Catégorie", "Etudiants", "Français"]][:-1]

In [53]:
df_expenses_cat.head()

,Catégorie,Etudiants,Français
0,Produits alimentaires et boissons non alcoolisées,16.768891,16.090613
1,Boissons alcoolisées et tabac,3.517955,2.875439
2,Articles d'habillement et chaussures,5.812929,5.012124
3,"Logement, eau, gaz, électricité et autres comb...",24.132486,16.251477
4,"Meubles, articles de ménage et entretien coura...",4.751010,5.684839


### Comparaison et mapping des catégories

In [54]:
df_operations["Catégorie"].unique() # toutes les catégories dans df_operations

array(['Téléphone', 'Jeux', 'Cadeaux', 'Restaurants', 'Transports',
       'Alimentation', 'Snacking', 'Cantine', 'Livres',
       'Mutuelle de santé', 'Loyer', 'Travaux', 'Pharmacie', 'Multimédia',
       'Evènements', 'Habillement', 'Autres dépenses', 'Sorties',
       'Frais bancaires', 'Ameublement/équipement', 'Santé/Bien être',
       "Retrait d'argent", 'Dons', 'Chaussures', 'Frais de scolarité',
       'Assurance logement', 'Frais immobilier'], dtype=object)

In [55]:
df_expenses_cat["Catégorie"].unique() # toutes les catégories dans df_expenses_cat

array(['Produits alimentaires et boissons non alcoolisées',
       'Boissons alcoolisées et tabac',
       "Articles d'habillement et chaussures",
       'Logement, eau, gaz, électricité et autres combustibles',
       'Meubles, articles de ménage et entretien courant de la maison',
       'Santé', 'Transports', 'Communications', 'Loisirs et culture',
       'Enseignement', 'Restauration et hôtels',
       'Biens et services divers (1)'], dtype=object)

In [56]:
categories_mapping_operations = {
    "Habitation":["Habitation", "Travaux", "Electricité, gaz, eau", "Impôts : taxe foncière", "Impôts : taxe habitation",
                 "Habitation secondaire", "Loyer", "Jardin", "Assurance logement", "Ménage/repassage",
                 "Ameublement/équipement", "Frais immobilier"],
    "Loisirs":["Loisirs", "Multimédia", "Vacances", "Animaux", "Sport", "Sorties", "Evènements", "Jeux",
              "Hôtels", "Livres", "Restaurants"],
    "Communications":["Téléphone", "Internet/TV", "Communications"],
    "Alimentation" :["Alimentation", "Cantine", "Snacking"],
    "Transports":["Transports", "Véhicule", "Moto", "Péages", "Essence", "Location Voiture", "Assurance auto",
                 "Assurance moto", "Parking"],
    "Santé/Bien être":["Santé/Bien être", "Soin du corps", "Services à la personne", "Mutuelle de santé", "Pharmacie",
                      "Assurance santé", "Aides à domicile", "Coiffeur"],
    "Habillement":["Habillement", "Chaussures", "Accessoires"],
    "Autres":["Frais Pro", "Enfants", "Amendes", "Frais de justice", "Frais de scolarité", "Gardes d'enfant", "Dons", 
             "Impôts : autres", "Assurances : autres", "Autres dépenses",
             "Impôts : revenus", "Epargne", "Prêts", "Agios", "Frais bancaires", "Assurance vie", "Bourse", 
             "Retrait d'argent", "Prêt relais", "Cadeaux"]
}
categories_mapping_operations = {subcat:cat for cat in categories_mapping_operations.keys() for subcat in categories_mapping_operations[cat]}

In [57]:
categories_mapping_expenses_cat = {
    'Produits alimentaires et boissons non alcoolisées':"Alimentation",
    'Boissons alcoolisées et tabac':"Alimentation",
    "Articles d'habillement et chaussures":"Habillement",
    'Logement, eau, gaz, électricité et autres combustibles':"Habitation",
    'Meubles, articles de ménage et entretien courant de la maison' :"Habitation",
    'Santé' : "Santé/Bien être",
    'Transports':"Transports",
    'Communications': "Communications",
    'Loisirs et culture':"Loisirs",
    'Enseignement':"Autres",
    'Restauration et hôtels':"Loisirs",
    'Biens et services divers (1)':"Autres"
}

### Transformation des données

In [58]:
df_operations = df_operations.rename(columns = {"Débit euros": "Montant"})
df_operations = df_operations.rename(columns = lambda col: ud.unidecode (col.lower ())) # noms des colonnes en minuscules et sans accents

In [59]:
df_operations.head ()

,libelle,date,montant,categorie
1,PRELEVEMENT \r\nBouygues Telecom,27/12/2021,11.99,Téléphone
2,PAIEMENT PAR CARTE \r\nLA FRANCAISE DES...,24/12/2021,8.50,Jeux
4,PAIEMENT PAR CARTE \r\nAMAZON EU SARL P...,23/12/2021,79.72,Cadeaux
5,PAIEMENT PAR CARTE \nFNAC LYON CEDEX 0 ...,20/12/2021,68.98,Cadeaux
6,PAIEMENT PAR CARTE \nGLOCAL DIFFUSION LUN...,20/12/2021,25.00,Cadeaux


In [60]:
df_operations["libelle"] = df_operations["libelle"].map (lambda x: x.splitlines ()[1]) # supprimer "PAIEMENT PAR CARTE"
df_operations["libelle"] = df_operations["libelle"].map (lambda x: re.split ("\d+", x) [0]) # supprimer les numéros de compte, etc
df_operations["libelle"] = df_operations["libelle"].map (lambda x: re.sub ("\s{2,}", "", x)) # supprimer les espaces en trop

In [61]:
df_operations["categorie"] = df_operations["categorie"].map(categories_mapping_operations) # on oublie les sous-catégories et on ne garde que les catégories
df_operations.head()

,libelle,date,montant,categorie
1,Bouygues Telecom,27/12/2021,11.99,Communications
2,LA FRANCAISE DES BOULOGNE,24/12/2021,8.50,Loisirs
4,AMAZON EU SARL PAYLI,23/12/2021,79.72,Autres
5,FNAC LYON CEDEX,20/12/2021,68.98,Autres
6,GLOCAL DIFFUSION LUNEL,20/12/2021,25.00,Autres


In [62]:
df_operations["categorie"].value_counts() # nombre d'occurences

Alimentation       134
Loisirs             64
Autres              54
Santé/Bien être     40
Transports          38
Habitation          19
Communications      13
Habillement          8
Name: categorie, dtype: int64

In [63]:
df_expenses_cat["Catégorie"] = df_expenses_cat["Catégorie"].map(categories_mapping_expenses_cat) # on transforme les données de sorte à avoir les mêmes catégories que l'autre jeu de données
df_expenses_cat = df_expenses_cat.groupby("Catégorie").sum().reset_index() # on rassemble les mêmes catégories
df_expenses_cat.head()

,Catégorie,Etudiants,Français
0,Alimentation,20.286845,18.966052
1,Autres,14.800799,16.625232
2,Communications,3.649845,2.711766
3,Habillement,5.812929,5.012124
4,Habitation,28.883495,21.936316


### Sauvegarde des données

In [64]:
df_operations.to_csv("../data/operations.csv", index=False)
df_expenses_cat.to_csv("../data/repartition_categories.csv", index=False)